In [35]:
import csv
import pandas as pd
import numpy as np
import os
from binance.client import Client
import time
import re
import requests
import gdax
from datetime import datetime
import string
import random
import coinbase
from coinbase.wallet.client import OAuthClient, Client


In [36]:
api_input = open("APIKeyBin.txt", "r")
codes = api_input.readlines()
codes = [code.rstrip('\n') for code in codes]
api_key = codes[0]
api_secret = codes[1]

FileNotFoundError: [Errno 2] No such file or directory: 'APIKeyBin.txt'

In [251]:
# Gather data, rename files as necessary
btc = pd.read_csv("BTCfills(12_21_17).csv")
eth = pd.read_csv("ETHfills(12_22_17).csv")
ltc = pd.read_csv("LTCfills(12_21_17).csv")
alts = pd.read_csv("OrderHistory.csv")

In [252]:
btc

,trade id,product,side,created at,size,size unit,price,fee,total,price/fee/total unit
0,23419416,BTC-USD,SELL,2017-11-08T17:28:41.965Z,0.059822,BTC,7800.00,0.000000,466.609260,USD
1,23500329,BTC-USD,BUY,2017-11-09T07:48:16.687Z,0.067610,BTC,7376.95,1.246883,-499.999964,USD
2,24375436,BTC-USD,SELL,2017-11-17T00:25:38.578Z,0.017521,BTC,7934.98,0.347578,138.683429,USD
3,25471432,BTC-USD,SELL,2017-11-30T01:54:35.492Z,0.033985,BTC,10484.00,0.000000,356.295071,USD
4,25852043,BTC-USD,BUY,2017-12-02T23:16:27.125Z,0.010000,BTC,11006.93,0.000000,-110.069300,USD
5,25852067,BTC-USD,SELL,2017-12-02T23:16:57.654Z,0.018604,BTC,11006.94,0.000000,204.778285,USD


In [253]:
btc_earnings = btc['total'].sum()

In [254]:
eth_earnings = eth['total'].sum()

In [255]:
ltc_earnings = ltc['total'].sum()

In [256]:
alts.head()

,Date,Pair,Type,Order Price,Order Amount,Avg Trading Price,Filled,Total,status
0,2017-12-21 05:55:06,MODBTC,SELL,0.0001562,102.21,0.0001563,102.210,0.015976,Filled
1,NaN,Date,Trading Price,Filled,Total,Fee,NaN,NaN,NaN
2,NaN,2017-12-21 05:55:06,0.0001563,102.21,0.01597542,0.00001598BTC,NaN,NaN,NaN
3,2017-12-20 08:56:43,XMRBTC,BUY,0.023801,0.692,0.023801,0.692,0.016471,Filled
4,NaN,Date,Trading Price,Filled,Total,Fee,NaN,NaN,NaN


In [257]:
alts_trans = alts.dropna(subset=['Date'])
alts_filled = alts_trans[alts_trans['status'] == 'Filled']
negated_sells = alts_filled.apply(lambda x: x.loc['Total'] if x.loc['Type'] == 'SELL' else -x.loc['Total'], axis=1).astype('float')
negated_coins = alts_filled.apply(lambda x: x.loc['Order Amount'] if x.loc['Type'] == 'SELL' else -float(x.loc['Order Amount']), axis=1).astype('float')
alts_filled.loc[:, 'Total'] = negated_sells
alts_filled.loc[:, 'Order Amount'] = negated_coins
buy_sell_amts = alts_filled.groupby(['Pair', 'Type'])[['Order Amount', 'Total']].sum()
buy_sell_amts

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Order Amount        Total
Pair    Type                           
ARKBTC  BUY     -35.000000    -0.015558
        SELL     34.960000     0.012928
BTCUSDT BUY      -0.124588 -1841.370000
        SELL      0.131097  1843.210000
ETHBTC  BUY      -0.945000    -0.032604
IOTABTC BUY    -300.000000    -0.050137
        SELL    299.000000     0.050318
LTCBTC  BUY      -4.690000    -0.072435
        SELL     10.070000     0.165576
MODBTC  BUY    -152.320000    -0.027642
        SELL    152.160000     0.028004
QTUMBTC BUY      -7.000000    -0.011697
        SELL      6.990000     0.012589
TRXBTC  SELL    500.000000     0.000130
WTCBTC  BUY     -15.000000    -0.012646
        SELL     14.000000     0.009114
XMRBTC  BUY      -2.182000    -0.046423
        SELL      0.544000     0.013056

In [258]:
profits = buy_sell_amts.groupby('Pair').sum()
profits = profits.filter(regex='BTC$', axis = 0)
curr_btc_price = float(input("Current BTC price in USD: ").replace(',', ''))
profits_in_usd = pd.DataFrame()
profits_in_usd['Amount of Coin in Binance'] = profits['Order Amount']
profits_in_usd['Total'] = profits['Total'].apply(lambda x: x * curr_btc_price)

Current BTC price in USD: 13000


In [259]:
profits_in_usd

,Amount of Coin in Binance,Total
Pair,,
ARKBTC,-0.040,-34.18090
ETHBTC,-0.945,-423.85200
IOTABTC,-1.000,2.35521
LTCBTC,5.380,1210.83300
MODBTC,-0.160,4.70470
QTUMBTC,-0.010,11.59600
TRXBTC,500.000,1.69000
WTCBTC,-1.000,-45.92159
XMRBTC,-1.638,-433.77100


In [260]:
alts_earnings = profits_in_usd.sum()

In [261]:
alts_earnings + btc_earnings + eth_earnings + ltc_earnings

Amount of Coin in Binance    874.466211
Total                        667.332631
dtype: float64

In [262]:
btc_earnings

556.2967804399215

In [263]:
eth_earnings

-314.509168702054

In [264]:
ltc_earnings

132.09159926903092

In [33]:
client = Client(api_key, api_secret)

NameError: name 'api_key' is not defined

In [32]:
client.get_deposit_history()

NameError: name 'client' is not defined

In [190]:
# get all trades from all symbols

all_symbols = client.get_all_tickers()
all_symbols = [coin['symbol'] for coin in all_symbols]

trades = []
for symbol in all_symbols:
    this_coins_trade = client.get_my_trades(symbol = symbol)
    for trade in this_coins_trade:
        trade['symbol'] = symbol
    trades.extend(this_coins_trade)
    time.sleep(0.1)


In [223]:
trades_df = pd.DataFrame(trades)
trades_df['coinOfInterest'] = trades_df['symbol'].apply(lambda x: re.sub(r'BTC$|USDT$', '', x))
coin_of_basis = trades_df.apply(lambda x: x['symbol'].replace(x['coinOfInterest'], ''), axis=1)
trades_df['coinOfBasis'] = coin_of_basis
trades_df

,commission,commissionAsset,id,isBestMatch,isBuyer,isMaker,orderId,price,qty,symbol,time,coinOfInterest,coinOfBasis
0,0.00046700,ETH,6158916,True,True,True,26691743,0.03450100,0.46700000,ETHBTC,1513088433121,ETH,BTC
1,0.00008000,ETH,6158917,True,True,True,26691743,0.03450100,0.08000000,ETHBTC,1513088433234,ETH,BTC
2,0.00008000,ETH,6158918,True,True,True,26691743,0.03450100,0.08000000,ETHBTC,1513088433333,ETH,BTC
3,0.00004100,ETH,6158919,True,True,True,26691743,0.03450100,0.04100000,ETHBTC,1513088433661,ETH,BTC
4,0.00015000,ETH,6158920,True,True,True,26691743,0.03450100,0.15000000,ETHBTC,1513088433721,ETH,BTC
5,0.00012700,ETH,6158921,True,True,True,26691743,0.03450100,0.12700000,ETHBTC,1513088434079,ETH,BTC
6,0.00001601,BTC,1815340,True,False,False,11156093,0.01469000,1.09000000,LTCBTC,1513076134922,LTC,BTC
7,0.00005450,BTC,1815344,True,False,True,11156093,0.01469000,3.71000000,LTCBTC,1513076135898,LTC,BTC
8,0.00228000,LTC,1815665,True,True,True,11156932,0.01459200,2.28000000,LTCBTC,1513076332906,LTC,BTC
9,0.00002449,BTC,1830042,True,False,True,11186707,0.01749500,1.40000000,LTCBTC,1513081852082,LTC,BTC


In [59]:
# TODO: automate keys creation (pull from product ticker or something)
tables = {'BTC': None, 'ETH': None, 'LTC': None}

In [8]:
api_input = open("APIKeyGDAX.txt", "r")
codes = api_input.readlines()
codes = [code.rstrip('\n') for code in codes]
GDAXapi_key = codes[0]
GDAXapi_secret = codes[1]
GDAXpass = codes[2]

In [9]:
auth_client = gdax.AuthenticatedClient(GDAXapi_key, GDAXapi_secret, GDAXpass)

In [10]:
request = auth_client.get_accounts()

In [11]:
holdings = pd.DataFrame(request)
holdings

,available,balance,currency,hold,id,profile_id
0,1686.8402310068971,1686.8402310068971000,USD,0.0000000000000000,498953be-01aa-4e2f-b942-645159b365ed,df592a89-8873-4436-a5d9-4d8410c8fdc0
1,1.5,1.5000000000000000,LTC,0.0000000000000000,519dad36-ad08-4f45-b41a-800330a92be8,df592a89-8873-4436-a5d9-4d8410c8fdc0
2,1.60274544,1.6027454400000000,ETH,0.0000000000000000,a41fd816-0d38-486d-9d39-8e520a6bf7f7,df592a89-8873-4436-a5d9-4d8410c8fdc0
3,0,0.0000000000000000,BTC,0.0000000000000000,f8abc92c-acd1-44cc-859a-e99328f2f63e,df592a89-8873-4436-a5d9-4d8410c8fdc0
4,0,0.0000000000000000,BCH,0.0000000000000000,12ed5862-38a9-45d5-89fd-d0486e09e7a1,df592a89-8873-4436-a5d9-4d8410c8fdc0


In [12]:
profile_ids = {account['currency']: account['id'] for account in request}

In [13]:
# fix this, doesn't make sense for files to start out empty
files = {}

In [56]:
profile_ids

{'BCH': '12ed5862-38a9-45d5-89fd-d0486e09e7a1',
 'BTC': 'f8abc92c-acd1-44cc-859a-e99328f2f63e',
 'ETH': 'a41fd816-0d38-486d-9d39-8e520a6bf7f7',
 'LTC': '519dad36-ad08-4f45-b41a-800330a92be8',
 'USD': '498953be-01aa-4e2f-b942-645159b365ed'}

In [25]:
auth_client.get_account_history(profile_ids['LTC'])

[[{'amount': '1.0000000000000000',
   'balance': '1.5000000000000000',
   'created_at': '2017-12-26T05:24:38.268715Z',
   'details': {'order_id': '90fdfa1c-f7a9-4733-9677-a591d7579c71',
    'product_id': 'LTC-USD',
    'trade_id': '20902470'},
   'id': 580750792,
   'type': 'match'},
  {'amount': '0.5000000000000000',
   'balance': '0.5000000000000000',
   'created_at': '2017-12-21T05:21:57.836593Z',
   'details': {'order_id': 'c026765a-4da1-4c2c-b9ea-a95e4ca2a16c',
    'product_id': 'LTC-USD',
    'trade_id': '19919431'},
   'id': 551486901,
   'type': 'match'},
  {'amount': '-0.9183654900000000',
   'balance': '0.0000000000000000',
   'created_at': '2017-12-21T04:39:03.360988Z',
   'details': {'order_id': 'ef61ac38-cea6-4328-9da7-3dfd72124abe',
    'product_id': 'LTC-USD',
    'trade_id': '19906618'},
   'id': 551259960,
   'type': 'match'},
  {'amount': '-2.3000000000000000',
   'balance': '0.9183654900000000',
   'created_at': '2017-12-15T15:25:48.359079Z',
   'details': {'order_id

In [53]:
fills = auth_client.get_fills()

In [54]:
fills

[[{'created_at': '2017-12-02T23:16:57.654Z',
   'fee': '0.0000000000000000',
   'liquidity': 'M',
   'order_id': '7be5285a-9909-46bf-b59f-64377b22af25',
   'price': '11006.94000000',
   'product_id': 'BTC-USD',
   'profile_id': 'df592a89-8873-4436-a5d9-4d8410c8fdc0',
   'settled': True,
   'side': 'sell',
   'size': '0.01860447',
   'trade_id': 25852067,
   'usd_volume': '204.7782850218000000',
   'user_id': '59b38107aebae700d82d9ffa'},
  {'created_at': '2017-12-02T23:16:27.125Z',
   'fee': '0.0000000000000000',
   'liquidity': 'M',
   'order_id': 'fa8879fa-6289-4b80-9493-19717277205b',
   'price': '11006.93000000',
   'product_id': 'BTC-USD',
   'profile_id': 'df592a89-8873-4436-a5d9-4d8410c8fdc0',
   'settled': True,
   'side': 'buy',
   'size': '0.01000000',
   'trade_id': 25852043,
   'usd_volume': '110.0693000000000000',
   'user_id': '59b38107aebae700d82d9ffa'},
  {'created_at': '2017-11-30T01:54:35.492Z',
   'fee': '0.0000000000000000',
   'liquidity': 'M',
   'order_id': '1e107

In [27]:
fills_df = pd.DataFrame(fills[0])
fills_df['created_at'] = fills_df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
fills_df = fills_df.sort_values(by='created_at').reset_index(drop=True)

In [ ]:
fills_df

In [89]:
def get_average_price(coin, table):
    #checks
    table = table.sort_values(by='created_at').reset_index(drop=True)
    unique_coins = table['product_id'].unique()
    unique_coins = [str(coin_ex) for coin_ex in unique_coins]
    if 'nan' in unique_coins and len(unique_coins) != 2:
        print("More than one currency in average price check")
        return
    elif 'nan' not in unique_coins and len(unique_coins) != 1:
        print("More than one currency in average price check")
        return
    start_index = 0
    table['price'] = table['price'].astype(float)
    table['size'] = table['size'].astype(float)
    if 'rolling_average' in table.columns:
        index = table['rolling_average'].index[table['rolling_average'].apply(np.isnan)]
        table_index = table.index.values.tolist()
        start_index = table_index.index(index[0])
    if start_index == 0:
        table['rolling_average'] = pd.Series(table.loc[0, 'price'])
        table['rolling_stash'] = pd.Series(table.loc[0, 'size'])
        start_index += 1
    curr_price_index = start_index
    while curr_price_index < len(table):
        if table['type'][curr_price_index] == 'buy' or table['type'][curr_price_index] == 'deposit':
            prev_price = table['rolling_average'][curr_price_index-1]
            prev_stash = table['rolling_stash'][curr_price_index-1]
            this_price = table['price'][curr_price_index]
            this_stash = table['size'][curr_price_index]
            total_stash = prev_stash + this_stash
            table.loc[curr_price_index, 'rolling_average'] = (prev_price*(prev_stash/total_stash)) + (this_price*(this_stash/total_stash))
            table.loc[curr_price_index, 'rolling_stash'] = total_stash
        else:
            remove_stash = table['size'][curr_price_index]
            table.loc[curr_price_index, 'rolling_average'] = table['rolling_average'][curr_price_index-1]
            table.loc[curr_price_index, 'rolling_stash'] = table['rolling_stash'][curr_price_index-1] + remove_stash
        curr_price_index += 1
    tables[coin] = table
    return table

In [171]:
# pass in fills of one type of coin
# coin here is coin
def update_fills_table(coin):
    fills = auth_client.get_fills(product_id=coin + '-USD')
    old_table = None
    fills_list = []
    for fill_page in fills:
        fills_list.extend(fill_page)
    fills_df = pd.DataFrame(fills_list)
    fills_df.loc[:, 'created_at'] = fills_df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
    if tables[coin] is not None:
        old_table = tables[coin]
        latest_date = get_latest_date(old_table)
        fills_df = fills_df[fills_df['created_at'] > latest_date]
    fills_df = fills_df.sort_values(by='created_at').reset_index(drop=True)
#     product_id = coin + '-USD'
#     coin_chart = fills_df[fills_df['product_id'] == product_id].reset_index(drop=True)
    fills_df.rename(columns = {'side':'type'}, inplace = True)
    fills_df['size'] = fills_df['size'].astype(float)
    fills_df['fee'] = fills_df['fee'].astype(float)
    fills_df['price'] = fills_df['price'].astype(float)
    fills_df['usd_volume'] = fills_df['usd_volume'].astype(float)
    fills_df['trade_id'] = fills_df['trade_id'].astype(str)
    negated_sells = fills_df.apply(lambda x: -x.loc['size'] if x.loc['type'] == 'sell' else x.loc['size'], axis=1).astype('float')
    fills_df['size'] = negated_sells
    return fills_df

In [152]:
# pass in transfers of one type of coin
# coin here is just the coin (e.g. 'ETH')
def update_transfers_table(coin):
    account_history = auth_client.get_account_history(profile_ids[coin])
    combined_list = []
    for account_page in account_history:
        combined_list.extend(account_page)
    coin_hist = pd.DataFrame(combined_list)
    coin_transfers = coin_hist[coin_hist['type'] == 'transfer']
    coin_transfers.loc[:, 'created_at'] = coin_transfers['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
    
    if tables[coin] is not None:
        old_table = tables[coin]
        latest_date = get_latest_date(old_table)
        coin_transfers = coin_transfers[coin_transfers['created_at'] > latest_date]
    coin_transfers.loc[:, 'transfer_id'] = coin_transfers['details'].apply(lambda x: x.get('transfer_id'))
    coin_transfers.rename(columns = {'amount': 'size'}, inplace=True)
    coin_transfers.loc[:, 'id'] = coin_transfers['id'].astype(str)
    coin_transfers.loc[:, 'size'] = coin_transfers['size'].astype(float)
    coin_transfers = coin_transfers.reset_index(drop=True)
    prices = []
    types = []
    for index, row in coin_transfers.iterrows():
        if row['size'] < 0:
            cost = float(input("What is the loss (in USD) of the {0} {1} for the withdrawal at {2}".format(-row['size'], coin, row['created_at'])))
            price = cost/(-row['size'])
            prices.append(price)
            types.append('withdrawal')
        else:
            cost = float(input("What is the gain (in USD) of the {0} {1} for the deposit at {2}".format(row['size'], coin, row['created_at'])))
            price = cost/row['size']
            prices.append(price)
            types.append('deposit')
    prices_series = pd.Series(prices)
    types_series = pd.Series(types)
    coin_transfers.loc[:, 'price'] = prices_series
    coin_transfers.loc[:, 'type'] = types_series
    return coin_transfers

In [92]:
def get_latest_date(table):
    sorted_table = table.sort_values(by='created_at')
    return sorted_table.tail(1)['created_at'].values[0]

In [93]:
def update_all_fills():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-93-f3a61c4357cd>, line 2)

In [94]:
def update_all_transfers():

SyntaxError: unexpected EOF while parsing (<ipython-input-94-c7e365348058>, line 1)

In [95]:
def append_transfers(transfers, main_table):
    transfers = transfers.loc[:, ['size', 'created_at', 'id', 'type', 'transfer_id', 'price']]
    main = pd.concat([main_table, transfers], ignore_index=True)
    main = main.sort_values(by='created_at').reset_index(drop=True)
    return main

In [96]:
def drop_nonessential(table):
    columns_to_drop = set(['order_id', 'id', 'trade_id', 'user_id', 'liquidity', 'profile_id', 'transfer_id'])
    columns_exist = columns_to_drop.intersection(table.columns)
    if list(columns_exist) != []:
        dropped_table = table.drop(list(columns_exist), axis=1)
    return dropped_table

In [139]:
def create_csv(table):
    coin = get_product_id(table)
    name = coin+'_summary.csv'
    table.to_csv(name, index=False)
    files[coin] = name
    

In [98]:
def get_product_id(table):
    coins = table['product_id'].unique()
    clean_coins = [x for x in coins if str(x) != 'nan']
    if len(clean_coins) > 1:
        print("More than one coin in table being saved.")
    if len(clean_coins) == 0:
        print("No coins in table")
    coin = clean_coins[0]
    return coin

In [99]:
def get_gains(table):
    sells_withdraws = table[(table['type'] == 'sell') | (table['type'] == 'withdraw')]
    price_diff = (sells_withdraws['price'] - sells_withdraws['rolling_average'])*np.negative(sells_withdraws['size'])
    return price_diff.sum()

In [100]:
def gains_if_sell_now(table):
    product_id = get_product_id(table)
    price = float(auth_client.get_product_ticker(product_id)['price'])
    gains = get_gains(table)
    flush_out = (price - table.tail(1)['rolling_average'].values[0]) * (table.tail(1)['rolling_stash'].values[0])
    current_gains = gains + flush_out
    return current_gains

In [164]:
def set_types(table):
    table['created_at'] = pd.to_datetime(table['created_at'])
    table['trade_id'] = table['trade_id'].astype(str)
    table['id'] = table['id'].astype(str)

    return table

In [35]:
BTC_transfers = create_transfers_table('BTC')
BTC_fills = create_fills_table('BTC', fills)
BTC_table = append_transfers(BTC_transfers, BTC_fills)
BTC_avg = get_average_price(BTC_table)

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

What is the gain (in USD) of the 0.00860447 BTC for the deposit at 2017-12-02 23:08:49.62883494.56
What is the gain (in USD) of the 0.03398465 BTC for the deposit at 2017-11-30 01:53:55.549125351.53
What is the gain (in USD) of the 0.01752128 BTC for the deposit at 2017-11-16 23:43:01.850298137.19
What is the loss (in USD) of the 0.06760966 BTC for the withdrawal at 2017-11-09 07:57:18.148940499.20
What is the gain (in USD) of the 0.0598217 BTC for the deposit at 2017-09-16 05:38:08.130303234.73


In [36]:
LTC_transfers = create_transfers_table('LTC')
LTC_fills = create_fills_table('LTC', fills)
LTC_table = append_transfers(LTC_transfers, LTC_fills)
LTC_table = LTC_table.sort_values(by='created_at').reset_index(drop=True)
LTC_avg = get_average_price(LTC_table)

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

What is the gain (in USD) of the 3.21836549 LTC for the deposit at 2017-12-14 06:24:27.3902151016.10
What is the loss (in USD) of the 6.21936549 LTC for the withdrawal at 2017-12-14 04:06:15.2740971959.10
What is the gain (in USD) of the 2.41431 LTC for the deposit at 2017-12-12 14:44:14.110648913.71
What is the loss (in USD) of the 4.8 LTC for the withdrawal at 2017-12-12 10:18:31.2233021188.09
What is the loss (in USD) of the 2.0 LTC for the withdrawal at 2017-12-12 09:53:49.121355502.72
What is the gain (in USD) of the 17.53564497 LTC for the deposit at 2017-11-20 01:39:10.4336581267.82
What is the gain (in USD) of the 2.96160742 LTC for the deposit at 2017-09-16 05:39:19.277842169.52


In [37]:
ETH_transfers = create_transfers_table('ETH')
ETH_fills = create_fills_table('ETH', fills)
ETH_table = append_transfers(ETH_transfers, ETH_fills)
ETH_table = ETH_table.sort_values(by='created_at').reset_index(drop=True)
ETH_avg = get_average_price(ETH_table)

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

What is the gain (in USD) of the 0.934055 ETH for the deposit at 2017-12-12 14:49:01.301000549.36
What is the gain (in USD) of the 0.42434979 ETH for the deposit at 2017-09-16 05:38:51.368638116.22


In [43]:
drop_nonessential(BTC_avg)

,created_at,fee,price,product_id,settled,size,type,usd_volume,rolling_average,rolling_stash
0,2017-09-16 05:38:08.130303,NaN,3923.826972,NaN,NaN,0.059822,deposit,NaN,3923.826972,5.982170e-02
1,2017-11-08 17:28:41.965000,0.0000000000000000,7800.000000,BTC-USD,True,-0.059822,sell,None,3923.826972,0.000000e+00
2,2017-11-09 07:48:16.687000,1.2468827033425000,7376.950000,BTC-USD,True,0.067610,buy,None,7376.950000,6.760966e-02
3,2017-11-09 07:57:18.148940,NaN,7383.560278,NaN,NaN,-0.067610,withdrawal,NaN,7376.950000,0.000000e+00
4,2017-11-16 23:43:01.850298,NaN,7829.907404,NaN,NaN,0.017521,deposit,NaN,7829.907404,1.752128e-02
5,2017-11-17 00:25:38.578000,0.3475775159360000,7934.980000,BTC-USD,True,-0.017521,sell,139.0310063744000000,7829.907404,0.000000e+00
6,2017-11-30 01:53:55.549125,NaN,10343.787563,NaN,NaN,0.033985,deposit,NaN,10343.787563,3.398465e-02
7,2017-11-30 01:54:35.492000,0.0000000000000000,10484.000000,BTC-USD,True,-0.033985,sell,356.2950706000000000,10343.787563,0.000000e+00
8,2017-12-02 23:08:49.628834,NaN,10989.636782,NaN,NaN,0.008604,deposit,NaN,10989.636782,8.604470e-03
9,2017-12-02 23:16:27.125000,0.0000000000000000,11006.930000,BTC-USD,True,0.010000,buy,110.0693000000000000,10998.931977,1.860447e-02


In [38]:
get_gains(BTC_avg)

238.6343219962

In [39]:
get_gains(LTC_avg)

673.581715628598

In [40]:
get_gains(ETH_avg)

767.3386054800826

In [179]:
tables['BTC'] = set_types(pd.read_csv('BTC-USD_summary.csv'))
tables['LTC'] = set_types(pd.read_csv('LTC-USD_summary.csv'))
tables['ETH'] = set_types(pd.read_csv('ETH-USD_summary.csv'))

In [180]:
update_fills = update_fills_table('ETH')
update_transfers = update_transfers_table('ETH')
update_table = append_transfers(update_transfers, update_fills)
new_table = pd.concat([tables['ETH'], update_table])
new_table = new_table.sort_values(by='created_at').reset_index(drop=True)
tables['ETH'] = get_average_price('ETH', new_table)

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [182]:
for table in tables.values():
    create_csv(table)

In [159]:
new_table['fee'].values[-1]

0.0

In [183]:
BTC_price = gains_if_sell_now(tables['BTC'])
LTC_price = gains_if_sell_now(tables['LTC'])
ETH_price = gains_if_sell_now(tables['ETH'])
BTC_price + LTC_price + ETH_price

1627.298900349698

In [ ]:
BTC_update = auth_client.get_account_history()

In [85]:
BTC_types = set_types(BTC_avg)

numpy.datetime64('2017-09-16T05:38:08.130303000')

In [2]:
coinbase_input = open("APIKeyCoinbase.txt", "r")
codes = coinbase_input.readlines()
codes = [code.rstrip('\n') for code in codes]
coinbase_api_key = codes[0]
client_id = codes[1]
client_secret = codes[2]

In [ ]:
client = Client(api_key, api_secret)


In [ ]:
access_token = input("Access token: ")
# refresh_token = input("Refresh token: ")
client = OAuthClient(access_token, '')

In [ ]:
client.get_accounts()

In [15]:
random_string = ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(15))

coinbase_input_data = {}
coinbase_input_data['reponse_type'] = 'code'
coinbase_input_data['client_id'] = client_id
coinbase_input_data['redirect_uri'] = 'http://jackyzhu.me'
coinbase_input_data['state'] = random_string
coinbase_input_data['scope'] = 'wallet:accounts:read'

In [16]:
user_response = requests.get('https://www.coinbase.com/oauth/authorize', coinbase_input_data)

In [17]:
random_string

'YpUkOaninqmq3qy'

In [18]:
user_response

<Response [200]>

In [19]:
user_response.request.url

'https://www.coinbase.com/oauth/authorize?reponse_type=code&client_id=f2030dd8f08e51f9b0738e06f9c89cbad9355d1ff8b4d60ae0dde53b7ec540ba&redirect_uri=http%3A%2F%2Fjackyzhu.me&state=YpUkOaninqmq3qy&scope=wallet%3Aaccounts%3Aread'